# FINAL

## IS 597 MLC


In [278]:
import pandas as pd
import nfl_data_py as nfl
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

In [3]:
nfl.see_weekly_cols()

Index(['player_id', 'player_name', 'player_display_name', 'position',
       'position_group', 'headshot_url', 'recent_team', 'season', 'week',
       'season_type', 'opponent_team', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points

In [4]:
# df = nfl.import_weekly_data([2001], ['player_id', 'player_name', 'player_display_name', 'position',
# 																										'position_group', 'headshot_url', 'season', 'week',
# 																										'season_type', 'opponent_team', 'completions', 'attempts',
# 																										'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'passing_air_yards',
# 																										'passing_yards_after_catch',  'carries', 'rushing_yards',
# 																										'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
# 																										'receptions', 'targets', 'receiving_yards', 'receiving_tds','receiving_air_yards',
# 																										'receiving_yards_after_catch', 'receiving_first_downs', 
# 																										'fantasy_points', 'fantasy_points_ppr'])

In [5]:
df = nfl.import_weekly_data([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024], ['player_display_name', 'position',
																										'position_group', 'season', 'week',
																										'season_type', 'opponent_team', 'completions', 'attempts',
																										'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'passing_air_yards',
																										'passing_yards_after_catch',  'carries', 'rushing_yards',
																										'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
																										'receptions', 'targets', 'receiving_yards', 'receiving_tds','receiving_air_yards',
																										'receiving_yards_after_catch', 'receiving_first_downs', 
																										'fantasy_points', 'fantasy_points_ppr'])

Downcasting floats.


In [430]:
df.head()

,player_display_name,position,position_group,season,week,season_type,opponent_team,completions,attempts,passing_yards,...,rushing_fumbles_lost,receptions,targets,receiving_yards,receiving_tds,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,fantasy_points,fantasy_points_ppr
0,Abdul-Karim al-Jabbar,RB,RB,2000,1,REG,KC,0,0,0.0,...,0.0,0,0,0.0,0,0.0,0.0,0.0,-0.2,-0.2
1,Rabih Abdullah,RB,RB,2000,1,REG,NE,0,0,0.0,...,0.0,1,1,3.0,0,0.0,0.0,0.0,0.3,1.3
2,Rabih Abdullah,RB,RB,2000,12,REG,CHI,0,0,0.0,...,0.0,0,0,0.0,0,0.0,0.0,0.0,1.8,1.8
3,Rabih Abdullah,RB,RB,2000,14,REG,DAL,0,0,0.0,...,0.0,1,1,11.0,0,0.0,0.0,1.0,4.9,5.9
4,Rabih Abdullah,RB,RB,2000,15,REG,MIA,0,0,0.0,...,0.0,0,0,0.0,0,0.0,0.0,0.0,1.4,1.4


In [431]:
df.shape

(127930, 29)

In [432]:
rb_data = df[df['position'] == 'RB']

In [433]:
wr_data = df[(df['position'] == 'WR') | (df['position'] == ' TE')]

In [434]:
qb_data = df[df['position'] == 'QB']

### Proof of concept


In [435]:
qb_data[(qb_data['player_display_name'] == 'Troy Aikman') & (qb_data['season'] == 2000)][['week', 'completions', 'attempts', 'passing_yards']]

,week,completions,attempts,passing_yards
7,1,0,5,0.0
8,4,15,26,197.0
9,5,15,23,131.0
10,7,20,40,201.0
11,8,9,15,154.0
12,9,4,5,43.0
13,11,23,36,302.0
14,12,19,33,138.0
15,13,30,43,276.0
16,14,16,30,158.0


In [436]:
lamar = qb_data[(qb_data['player_display_name'] == 'Lamar Jackson') & (qb_data['season'] == 2024)][['week', 'completions', 'attempts', 'passing_yards']]
lamar

,week,completions,attempts,passing_yards
1049,1,26,41,273.0
1050,2,21,34,247.0
1051,3,12,15,182.0
1052,4,13,18,156.0
1053,5,26,42,348.0
1054,6,20,26,323.0
1055,7,17,22,281.0
1056,8,23,38,289.0
1057,9,16,19,280.0
1058,10,25,33,290.0


In [437]:
# shifted_df = lamar[['completions', 'attempts', 'passing_yards']].shift(fill_value=0)

# lamar[['completions', 'attempts', 'passing_yards']] = shifted_df
# qb_data[['completions', 'attempts', 'passing_yards']] = (
#     qb_data.groupby(['player_display_name', 'season'])[['completions', 'attempts', 'passing_yards']]
#     .shift(fill_value=0)
# )

# qb_data[(qb_data['player_display_name'] == 'Lamar Jackson') & (qb_data['season'] == 2024)]

### **FUNCTIONS**


In [438]:
def preprocess_data(df, cols):
  df = df.reset_index()
  df[cols] = (df.groupby(['player_display_name', 'season'])[cols].shift(fill_value=0))
  return df

In [439]:
def pipeline(df, x_data, y_data, alpha=.1):
	X = df[x_data]
	y = df[y_data]

	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

	testing_rows = X_test.index
	testing_players = df.loc[testing_rows, 'player_display_name']
	

	lasso = Lasso(alpha=alpha)
	lasso.fit(X_train, y_train)
	y_pred = lasso.predict(X_test)

	mse = mean_squared_error(y_test, y_pred)
	r_squared = r2_score(y_test, y_pred)
	print(f"Mean Squared Error: {mse}")
	print(f"R-squared: {r_squared}")
	print("\n")
	for i in range(len(x_data)):
		print(x_data[i], lasso.coef_[i])

	print("\n")
	comparison_df = pd.DataFrame({'Player': testing_players, 'Actual': y_test, 'Predicted': y_pred})

	return lasso, comparison_df

## Quarterback Data


In [440]:
qb_x = [
			'completions', 'attempts',
			'passing_yards', 'passing_tds', 'interceptions', 'sacks',
			'passing_air_yards', 'passing_yards_after_catch', 'carries',
			'rushing_yards', 'rushing_tds', 'rushing_fumbles',
			'rushing_fumbles_lost', 'receptions', 'targets', 'receiving_yards',
			'receiving_tds', 'receiving_air_yards', 'receiving_yards_after_catch',
			'receiving_first_downs']

qb_y = 'fantasy_points_ppr'

In [441]:
qb_data = qb_data.reset_index()

In [442]:
qb_data[qb_x] = (
    qb_data.groupby(['player_display_name', 'season'])[qb_x]
    .shift(fill_value=0)
)

In [443]:
qb_data

,index,player_display_name,position,position_group,season,week,season_type,opponent_team,completions,attempts,...,rushing_fumbles_lost,receptions,targets,receiving_yards,receiving_tds,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,fantasy_points,fantasy_points_ppr
0,7,Troy Aikman,QB,QB,2000,1,REG,PHI,0,0,...,0.0,0,0,0.0,0,0.0,0.0,0.0,-2.000000,-2.000000
1,8,Troy Aikman,QB,QB,2000,4,REG,SF,0,5,...,0.0,0,0,0.0,0,0.0,0.0,0.0,7.880000,7.880000
2,9,Troy Aikman,QB,QB,2000,5,REG,CAR,15,26,...,0.0,0,0,0.0,0,0.0,0.0,0.0,3.440000,3.440000
3,10,Troy Aikman,QB,QB,2000,7,REG,NYG,15,23,...,0.0,0,0,0.0,0,0.0,0.0,0.0,2.740000,2.740000
4,11,Troy Aikman,QB,QB,2000,8,REG,ARI,20,40,...,0.0,0,0,0.0,0,0.0,0.0,0.0,14.160000,14.160000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15922,4054,Caleb Williams,QB,QB,2024,10,REG,NE,22,41,...,0.0,0,0,0.0,0,0.0,0.0,0.0,6.300000,6.300000
15923,4055,Caleb Williams,QB,QB,2024,11,REG,GB,16,30,...,0.0,0,0,0.0,0,0.0,0.0,0.0,16.240000,16.240000
15924,4056,Caleb Williams,QB,QB,2024,12,REG,MIN,23,31,...,0.0,0,0,0.0,0,0.0,0.0,0.0,26.900000,26.900000
15925,4057,Caleb Williams,QB,QB,2024,13,REG,DET,32,47,...,0.0,0,0,0.0,0,0.0,0.0,0.0,26.139999,26.139999


In [444]:
# pre_X = qb_data[qb_x]

# X = pd.concat([pre_X, pd.get_dummies(pre_X['opponent_team'])], axis=1)

In [445]:
X = qb_data[qb_x]
y = qb_data[qb_y]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

testing_rows = X_test.index
testing_players = qb_data.loc[testing_rows, 'player_display_name']
print("Players used in the testing set:\n", testing_players)

Players used in the testing set:
 11671      Russell Wilson
13451         Brian Hoyer
13074      Deshaun Watson
11427           Tom Brady
3713            Jim Sorgi
               ...       
8398           Drew Brees
780      Daunte Culpepper
5975        David Garrard
12963          Derek Carr
14667          Sam Howell
Name: player_display_name, Length: 3186, dtype: object


Using cumsum() takes about 20x longer to run


In [446]:
lasso = Lasso(alpha=0.1)

lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_test)

In [447]:
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r_squared}")
print(f"Coefficients: {lasso.coef_}")

Mean Squared Error: 63.657692594648026
R-squared: 0.17748697168273053
Coefficients: [ 0.14743398 -0.02265299  0.00812483  0.51783616 -0.28371825 -0.
  0.00219539  0.01066665  0.24282114  0.03803105  0.         -0.
 -0.         -0.         -0.         -0.          0.         -0.
 -0.04184184 -0.        ]


In [448]:
comparison_df = pd.DataFrame({'Player': testing_players, 'Actual': y_test, 'Predicted': y_pred})
comparison_df

,Player,Actual,Predicted
11671,Russell Wilson,8.980000,14.681721
13451,Brian Hoyer,-0.300000,9.121860
13074,Deshaun Watson,20.900000,14.029354
11427,Tom Brady,26.000000,10.579115
3713,Jim Sorgi,-0.300000,7.009503
...,...,...,...
8398,Drew Brees,31.680000,16.606370
780,Daunte Culpepper,24.860001,16.419454
5975,David Garrard,17.940001,17.024589
12963,Derek Carr,0.600000,16.332648


In [449]:
for i in range(len(qb_x)):
  print(qb_x[i], lasso.coef_[i])

completions 0.14743398064730343
attempts -0.022652985835637282
passing_yards 0.00812483085937964
passing_tds 0.5178361586630509
interceptions -0.2837182469513504
sacks -0.0
passing_air_yards 0.0021953870093109414
passing_yards_after_catch 0.010666653686347585
carries 0.24282114489161674
rushing_yards 0.038031053931396684
rushing_tds 0.0
rushing_fumbles -0.0
rushing_fumbles_lost -0.0
receptions -0.0
targets -0.0
receiving_yards -0.0
receiving_tds 0.0
receiving_air_yards -0.0
receiving_yards_after_catch -0.04184183553515356
receiving_first_downs -0.0


In [460]:
qb_x2 = ['completions', 'attempts', 'passing_tds', 'passing_yards', 'interceptions', 'passing_air_yards', 'carries', 
        'passing_yards_after_catch', 'carries', 'rushing_yards', 'receiving_yards_after_catch']
pipeline(qb_data, qb_x2, qb_y, alpha=.01)

Mean Squared Error: 62.29296650003212
R-squared: 0.17281886068544483


completions 0.18158922524248036
attempts -0.029167058450836607
passing_tds 0.6285725012446445
passing_yards 0.005812907000748856
interceptions -0.3732230912918909
passing_air_yards 0.0037003803659313933
carries 0.23943969435727097
passing_yards_after_catch 0.007754101354606369
carries 0.002503120680204845
rushing_yards 0.036492278276931356
receiving_yards_after_catch -0.05709404998296682




(Lasso(alpha=0.01),
                 Player     Actual  Predicted
 3153       Luke McCown   2.720000   8.721424
 9722        Alex Smith  13.420000  15.447790
 5415        Ken Dorsey   4.000000   6.697863
 6587        Drew Brees  20.719999  14.644330
 449          Ray Lucas   0.120000   6.966167
 ...                ...        ...        ...
 13360        Tom Brady   4.760000  19.841865
 1850   Chad Hutchinson   1.740000  11.238150
 6615     David Garrard  16.020000  15.593696
 9570    Johnny Manziel   1.580000  10.517759
 15358  Aidan O'Connell  25.920000  12.245253
 
 [3186 rows x 3 columns])

## Receiver/Tight End Data


In [450]:
wr_x = ['targets', 'receptions', 'receiving_yards', 
        'receiving_tds', 'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_first_downs', ]

wr_y = 'fantasy_points_ppr'

In [451]:
wr_data = preprocess_data(wr_data, wr_x)

In [452]:
pipeline(wr_data, wr_x, wr_y)

Mean Squared Error: 52.39401939549882
R-squared: 0.1288884118327751


targets 0.344278427359436
receptions 0.3117450359285449
receiving_yards 0.01898381473416084
receiving_tds 0.0
receiving_air_yards -0.000758153299188978
receiving_yards_after_catch 0.0015113374288905825
receiving_first_downs 0.13375616846886007




(Lasso(alpha=0.1),
                 Player  Actual  Predicted
 2121      Troy Edwards     6.5   6.248691
 29141    Albert Wilson     6.9   8.685398
 1478      Jason Tucker     0.0   7.977551
 14066    Legedu Naanee     4.0   7.372018
 14881  Michael Jenkins    13.2   5.560134
 ...                ...     ...        ...
 15295     Devin Hester     1.7   9.460890
 18789      Davone Bess    12.3  11.533205
 21646    Donald Driver     0.0   6.579969
 573       Ike Hilliard     6.0  17.165744
 19558     David Gettis     6.2   8.359178
 
 [9750 rows x 3 columns])

In [461]:
wr_x2 = ['targets', 'receptions', 'receiving_yards', 'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_first_downs']
pipeline(wr_data, wr_x2, wr_y, alpha=.01)

Mean Squared Error: 54.85393805098479
R-squared: 0.11601049730844393


targets 0.35423329973590856
receptions 0.3082684758826212
receiving_yards 0.015664726308849224
receiving_air_yards -0.001064300264430945
receiving_yards_after_catch 0.0014989061644957126
receiving_first_downs 0.22836490701747242




(Lasso(alpha=0.01),
                  Player  Actual  Predicted
 9689   Justin McCareins     9.1   8.167564
 28090     Pierre Garcon     6.5   9.167948
 25692       Steve Smith    13.1  15.419973
 33241       J.J. Nelson     0.0   7.720497
 40813     Kalif Raymond    12.8   7.332186
 ...                 ...     ...        ...
 42837    DeAndre Carter     1.0   6.919033
 10082     Jerheme Urban     2.1   5.492941
 45915   Michael Pittman    19.5  17.519554
 18223  Devery Henderson    13.7   6.642268
 21872   Michael Jenkins     6.5   7.220137
 
 [9750 rows x 3 columns])

## Running Back Data


In [453]:
rb_data.columns

Index(['player_display_name', 'position', 'position_group', 'season', 'week',
       'season_type', 'opponent_team', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks',
       'passing_air_yards', 'passing_yards_after_catch', 'carries',
       'rushing_yards', 'rushing_tds', 'rushing_fumbles',
       'rushing_fumbles_lost', 'receptions', 'targets', 'receiving_yards',
       'receiving_tds', 'receiving_air_yards', 'receiving_yards_after_catch',
       'receiving_first_downs', 'fantasy_points', 'fantasy_points_ppr'],
      dtype='object')

In [454]:
rb_x = ['season', 'week', 'carries',
				'rushing_yards', 'rushing_tds', 'rushing_fumbles',
				'rushing_fumbles_lost', 'receptions', 'targets', 'receiving_yards',
				'receiving_tds', 'receiving_air_yards', 'receiving_yards_after_catch',
				'receiving_first_downs']
rb_y = 'fantasy_points_ppr'

In [455]:
rb_data = preprocess_data(rb_data, rb_x)
rb_data

,index,player_display_name,position,position_group,season,week,season_type,opponent_team,completions,attempts,...,rushing_fumbles_lost,receptions,targets,receiving_yards,receiving_tds,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,fantasy_points,fantasy_points_ppr
0,0,Abdul-Karim al-Jabbar,RB,RB,0,0,REG,KC,0,0,...,0.0,0,0,0.0,0,0.0,0.0,0.0,-0.2,-0.2
1,1,Rabih Abdullah,RB,RB,0,0,REG,NE,0,0,...,0.0,0,0,0.0,0,0.0,0.0,0.0,0.3,1.3
2,2,Rabih Abdullah,RB,RB,2000,1,REG,CHI,0,0,...,0.0,1,1,3.0,0,0.0,0.0,0.0,1.8,1.8
3,3,Rabih Abdullah,RB,RB,2000,12,REG,DAL,0,0,...,0.0,0,0,0.0,0,0.0,0.0,0.0,4.9,5.9
4,4,Rabih Abdullah,RB,RB,2000,14,REG,MIA,0,0,...,0.0,1,1,11.0,0,0.0,0.0,1.0,1.4,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33081,4083,Trey Benson,RB,RB,2024,8,REG,CHI,0,0,...,0.0,0,0,0.0,0,0.0,0.0,0.0,11.5,12.5
33082,4084,Trey Benson,RB,RB,2024,9,REG,NYJ,0,0,...,0.0,1,1,18.0,0,4.0,14.0,1.0,8.7,10.7
33083,4085,Trey Benson,RB,RB,2024,10,REG,SEA,0,0,...,0.0,2,2,25.0,0,-8.0,33.0,2.0,1.8,1.8
33084,4086,Trey Benson,RB,RB,2024,12,REG,MIN,0,0,...,0.0,0,0,0.0,0,0.0,0.0,0.0,2.0,2.0


In [456]:
pipeline(rb_data, rb_x, rb_y)

Mean Squared Error: 54.25492040626604
R-squared: 0.2460646398062425


season -0.001555416002078665
week -0.0
carries 0.33816898574357773
rushing_yards 0.02111260080452733
rushing_tds 0.012232611314524439
rushing_fumbles -0.0
rushing_fumbles_lost 0.0
receptions 0.0
targets 0.5713340706714972
receiving_yards 0.03389436192303849
receiving_tds 0.0
receiving_air_yards -0.017956837364380585
receiving_yards_after_catch 0.0
receiving_first_downs 0.0




(Lasso(alpha=0.1),
                   Player  Actual  Predicted
 1832         Lamar Smith     2.9  16.434621
 17487  LeGarrette Blount    10.4   4.292944
 5164        Shawn Bryson     5.2   4.049496
 13283      Cedric Benson     9.5  12.560719
 3680      Jonathan Wells    13.5   7.290796
 ...                  ...     ...        ...
 1895       Lamont Warren     3.5   8.497482
 11792    Brian Westbrook     1.3  15.604544
 27643    Jonathan Taylor    15.4   8.049556
 17860     Brandon Bolden    13.9   5.116703
 25386      Tevin Coleman     7.6   6.473194
 
 [6618 rows x 3 columns])

In [457]:
rb_x = ['carries',
				'rushing_yards', 'targets', 'receiving_yards', 'receiving_air_yards', 'receiving_yards_after_catch',]
rb_y = 'fantasy_points_ppr'

In [459]:
pipeline(rb_data, rb_x, rb_y, alpha=.01)

Mean Squared Error: 52.56596933877641
R-squared: 0.23725036284676337


carries 0.3033650642995945
rushing_yards 0.022270289120707602
targets 0.5161671294315807
receiving_yards 0.03247282517928695
receiving_air_yards -0.024664548892539526
receiving_yards_after_catch -0.003327846239506394




(Lasso(alpha=0.01),
                   Player     Actual  Predicted
 7352       Willie Parker  17.100000  16.487475
 19804      Chris Johnson   3.700000   4.153064
 16751     Brandon Bolden   6.400000  12.788013
 4173      Antowain Smith   1.000000  10.723803
 5381       Curtis Martin  25.400000  15.722337
 ...                  ...        ...        ...
 32277     Saquon Barkley  19.700001  21.132626
 9501      Chester Taylor  10.600000   7.016441
 8851         Reggie Bush  22.900000   9.957698
 7556       Cedric Benson   4.800000   8.906960
 14474  Cadillac Williams   0.400000   7.699257
 
 [6618 rows x 3 columns])